<a href="https://colab.research.google.com/github/Ucchwas/ECG-Classification-using-CNN/blob/main/RNN_Simple_Multiclass.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [29]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [30]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.metrics import classification_report
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score
from sklearn.metrics import confusion_matrix
from keras.utils.np_utils import to_categorical
from sklearn.utils import class_weight
import warnings
warnings.filterwarnings('ignore')

In [31]:
df = pd.read_csv('/content/drive/MyDrive/Datasets/mitbih_train_Ihua.csv')

In [32]:
df.drop('1',axis='columns',inplace=True)
#print(df.dtypes)
df.fillna(0, inplace=True)
X = df.drop('Label1',axis='columns')
#Xs = df.drop('Label1',axis='columns')
#X = Xs[df['Label1'] != 1 & 2 & 3]
y = testLabels = df.Label1.astype(np.float32)
#ys = testLabels = df.Label1.astype(np.float32)
#y = ys[df['Label1'] != 1 & 2 & 3]

In [33]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size= 0.2,train_size=0.8, random_state=15, stratify=y)
#print(y_train.value_counts())
#print(y.value_counts())
#print(y.value_counts())
#print(len(X_train.columns))

#from tensorflow_addons import losses
import tensorflow as tf
from tensorflow import keras
from sklearn.metrics import confusion_matrix , classification_report

In [34]:
from keras.layers import Input, Convolution1D, BatchNormalization, MaxPooling1D, Flatten, Dense
from keras.models import Model

In [35]:
def SimpleRNN_Model(X_train, y_train, X_test, y_test, loss, weights):
    # Convert DataFrame to NumPy arrays and reshape
    X_train = X_train.values.reshape(X_train.shape[0], 1, X_train.shape[1])
    X_test = X_test.values.reshape(X_test.shape[0], 1, X_test.shape[1])

    model = keras.Sequential([
        keras.layers.SimpleRNN(256, input_shape=(1, X_train.shape[2]), activation='relu'),
        keras.layers.Dense(256, activation='relu'),
        keras.layers.Dense(5, activation='softmax'),
    ])

    # compile the model
    model.compile(optimizer='Adam', loss=loss, metrics=['accuracy'])

    if weights == -1:
        model.fit(X_train, y_train, epochs=20)
    else:
        model.fit(X_train, y_train, epochs=20, class_weight=weights)

    print(model.evaluate(X_test, y_test))

    y_preds = model.predict(X_train)
    y_preds = np.argmax(y_preds, axis=1)

    print("Classification Report: \n", classification_report(y_train, y_preds))
    return model, y_preds


In [36]:
model, y_preds = SimpleRNN_Model(X_train, y_train, X_test, y_test, 'sparse_categorical_crossentropy', -1)
from sklearn.metrics import confusion_matrix
cm = confusion_matrix(y_train, y_preds)
print("Cconfusion Matrix: \n",cm)

Epoch 1/20
2189/2189 [==============================] - 10s 4ms/step - loss: 0.2246 - accuracy: 0.9369
Epoch 2/20
2189/2189 [==============================] - 10s 5ms/step - loss: 0.1375 - accuracy: 0.9610
Epoch 3/20
2189/2189 [==============================] - 8s 4ms/step - loss: 0.1146 - accuracy: 0.9676
Epoch 4/20
2189/2189 [==============================] - 9s 4ms/step - loss: 0.1009 - accuracy: 0.9709
Epoch 5/20
2189/2189 [==============================] - 9s 4ms/step - loss: 0.0888 - accuracy: 0.9735
Epoch 6/20
2189/2189 [==============================] - 8s 4ms/step - loss: 0.0826 - accuracy: 0.9753
Epoch 7/20
2189/2189 [==============================] - 10s 4ms/step - loss: 0.0761 - accuracy: 0.9765
Epoch 8/20
2189/2189 [==============================] - 11s 5ms/step - loss: 0.0696 - accuracy: 0.9785
Epoch 9/20
2189/2189 [==============================] - 8s 4ms/step - loss: 0.0648 - accuracy: 0.9801
Epoch 10/20
2189/2189 [==============================] - 9s 4ms/step - loss: 0

In [37]:
model.save("RNN_Simple_Multiclass.h5")

In [38]:
import tensorflow as tf

# Load the model
model = tf.keras.models.load_model("RNN_Simple_Multiclass.h5")

In [39]:
# Convert to TensorFlow Lite model
converter = tf.lite.TFLiteConverter.from_keras_model(model)
converter.target_spec.supported_ops = [tf.lite.OpsSet.TFLITE_BUILTINS, tf.lite.OpsSet.SELECT_TF_OPS]
converter._experimental_lower_tensor_list_ops = False
tflite_model = converter.convert()
open("RNN_Simple_Multiclass.tflite", "wb").write(tflite_model)

730700